In [1]:
import re
import datetime as dt 
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Dataset

In [2]:


# Load the dataset (Change this line if reading from SQL)
df = pd.read_excel(r"C:\Users\Rudra\Desktop\FP20 Analytics_Feb25\datasets\source\Transportation & Logistics Tracking Dataset.xlsx", sheet_name = 'Primary Data')  

warnings.filterwarnings("ignore")

In [3]:
# See all columns
df.columns

Index(['Gps Provider', 'Booking ID', 'Shipment Type', 'Booking Date',
       'Vehicle Registration', 'Origin Location', 'Destination Location',
       'Origin Location Latitude', 'Origin Location Longitude',
       'Destination Location Latitude', 'Destination Location Longitude',
       'Data Ping time', 'Planned ETA', 'Current Location', 'Actual ETA',
       'Current Location Latitude', 'Curren Location Longitude', 'Ontime',
       'Trip Start Date', 'Trip End Date', 'Transportation Distance (KM)',
       'Vehicle Type', 'Minimum Kms To Be Covered In A Day', 'Driver Name',
       'Driver Mobile No', 'Customer Name', 'Supplier Name',
       'Material Shipped'],
      dtype='object')

In [4]:
# Standardize column names: Strip spaces and convert to lowercase with underscores
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()


# Print updated column names for reference
print("Updated Column Names:", df.columns)


Updated Column Names: Index(['gps_provider', 'booking_id', 'shipment_type', 'booking_date',
       'vehicle_registration', 'origin_location', 'destination_location',
       'origin_location_latitude', 'origin_location_longitude',
       'destination_location_latitude', 'destination_location_longitude',
       'data_ping_time', 'planned_eta', 'current_location', 'actual_eta',
       'current_location_latitude', 'curren_location_longitude', 'ontime',
       'trip_start_date', 'trip_end_date', 'transportation_distance_(km)',
       'vehicle_type', 'minimum_kms_to_be_covered_in_a_day', 'driver_name',
       'driver_mobile_no', 'customer_name', 'supplier_name',
       'material_shipped'],
      dtype='object')


In [5]:
df.rename(columns={'curren_location_longitude': 'current_location_longitude'}, inplace=True)

In [6]:
# all columns details 
df.info(max_cols=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   gps_provider                        3585 non-null   object        
 1   booking_id                          3585 non-null   object        
 2   shipment_type                       3585 non-null   object        
 3   booking_date                        3585 non-null   datetime64[ns]
 4   vehicle_registration                3585 non-null   object        
 5   origin_location                     3585 non-null   object        
 6   destination_location                3585 non-null   object        
 7   origin_location_latitude            3585 non-null   float64       
 8   origin_location_longitude           3585 non-null   float64       
 9   destination_location_latitude       3585 non-null   float64       
 10  destination_location_lon

In [7]:
df.describe()

,booking_date,origin_location_latitude,origin_location_longitude,destination_location_latitude,destination_location_longitude,data_ping_time,current_location_latitude,current_location_longitude,transportation_distance_(km),minimum_kms_to_be_covered_in_a_day,driver_mobile_no
count,3585,3585.000000,3585.000000,3585.000000,3585.000000,3584,3584.000000,3584.000000,3437.000000,940.000000,2.561000e+03
mean,2020-06-21 21:10:33.783650560,17.870904,78.857500,19.049239,78.928819,2020-06-29 19:31:07.833147392,18.505396,78.947352,841.100320,250.531915,8.598002e+09
min,2019-04-15 15:15:13,9.973636,72.056000,8.172701,70.740636,2019-06-14 15:20:12,8.700890,69.657698,0.000000,250.000000,6.000546e+09
25%,2020-06-12 16:15:00,12.839000,76.835337,12.839000,76.835337,2020-06-25 14:52:38.750000128,12.835129,76.854271,107.000000,250.000000,7.657009e+09
50%,2020-07-23 13:50:18,16.560192,79.632000,18.660455,78.099421,2020-07-29 17:35:13,16.964161,78.209030,400.000000,250.000000,8.930201e+09
75%,2020-08-14 23:34:17.889999872,22.961777,80.184717,23.953847,79.975221,2020-08-18 00:05:08,23.283158,80.019062,1290.000000,250.000000,9.639269e+09
max,2020-12-03 13:10:21,30.000345,91.843582,32.684722,94.961065,2020-08-28 12:15:10,32.367928,95.529550,2898.000000,275.000000,1.000000e+10
std,NaN,5.688552,4.481210,6.390109,4.450874,NaN,6.130438,4.291132,851.889681,3.609543,1.132647e+09


In [8]:
df[['booking_date', 'data_ping_time', 'planned_eta', 'actual_eta', 'trip_start_date', 'trip_end_date']]

,booking_date,data_ping_time,planned_eta,actual_eta,trip_start_date,trip_end_date
0,2020-08-26 12:03:46.000,2020-08-28 12:15:10,2020-08-30 16:03:46,2020-08-28 12:48:07.380000,2020-08-26 16:16:00,2020-08-28 12:15:10
1,2020-08-27 15:21:48.570,2020-08-28 11:35:40,2020-08-31 19:21:48.570000,2020-08-28 12:13:51.543000,2020-08-27 15:21:06,2020-08-27 15:21:54.947000
2,2020-08-27 14:22:17.833,2020-08-28 11:25:38,2020-08-31 18:22:17.833000,2020-08-28 11:33:19.043000,2020-08-27 14:21:21,2020-08-27 14:22:25.037000
3,2020-08-28 00:32:20.523,2020-08-28 11:25:37,2020-09-01 04:32:20.523000,2020-08-28 11:31:29.780000,2020-08-28 00:31:41,2020-08-28 00:32:24.213000
4,2020-08-26 09:55:08.000,2020-08-28 08:35:08,2020-08-30 13:55:08,2020-08-29 07:09:41.173000,2020-08-26 15:35:00,2020-08-28 11:21:00
...,...,...,...,...,...,...
3580,2019-12-20 14:54:06.000,2019-12-16 10:45:15,2019-12-20 17:18:06,2019-12-22 01:51:12.667000,2019-12-20 14:54:06,2020-01-16 15:57:00
3581,2019-11-29 21:14:49.000,2019-12-12 09:05:13,2019-11-30 00:06:49,2019-12-18 15:52:54.543000,2019-11-29 21:14:49,2020-02-05 16:57:55
3582,2019-11-30 15:10:27.000,2019-12-05 17:50:15,2019-11-30 17:34:27,2019-12-11 03:36:19.660000,2019-11-30 15:10:27,2020-01-02 14:24:00
3583,2019-06-10 13:17:44.000,NaT,1899-12-30 08:58:00.000,1899-12-30 08:13:00.000,1899-12-30 00:00:00.000,1899-12-30 08:13:00.000


In [9]:
date_columns = ['booking_date','data_ping_time','planned_eta','actual_eta','trip_start_date','trip_end_date']

for col in date_columns:
    df[col] = pd.to_datetime(df[col])
    
df[date_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   booking_date     3585 non-null   datetime64[ns]
 1   data_ping_time   3584 non-null   datetime64[ns]
 2   planned_eta      3585 non-null   datetime64[ns]
 3   actual_eta       3560 non-null   datetime64[ns]
 4   trip_start_date  3585 non-null   datetime64[ns]
 5   trip_end_date    3585 non-null   datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 168.2 KB


In [10]:
df[date_columns]

for col in date_columns:
    df[col] = df[col].dt.strftime('%Y-%m-%d')

In [11]:
df[['booking_date', 'data_ping_time', 'planned_eta', 'actual_eta', 'trip_start_date', 'trip_end_date']]

,booking_date,data_ping_time,planned_eta,actual_eta,trip_start_date,trip_end_date
0,2020-08-26,2020-08-28,2020-08-30,2020-08-28,2020-08-26,2020-08-28
1,2020-08-27,2020-08-28,2020-08-31,2020-08-28,2020-08-27,2020-08-27
2,2020-08-27,2020-08-28,2020-08-31,2020-08-28,2020-08-27,2020-08-27
3,2020-08-28,2020-08-28,2020-09-01,2020-08-28,2020-08-28,2020-08-28
4,2020-08-26,2020-08-28,2020-08-30,2020-08-29,2020-08-26,2020-08-28
...,...,...,...,...,...,...
3580,2019-12-20,2019-12-16,2019-12-20,2019-12-22,2019-12-20,2020-01-16
3581,2019-11-29,2019-12-12,2019-11-30,2019-12-18,2019-11-29,2020-02-05
3582,2019-11-30,2019-12-05,2019-11-30,2019-12-11,2019-11-30,2020-01-02
3583,2019-06-10,NaN,1899-12-30,1899-12-30,1899-12-30,1899-12-30


# Break into Different Tables

In [12]:
# booking Tables 12 cols
booking_df = df[[    
    "booking_id", "shipment_type", "booking_date", 
    "planned_eta", "actual_eta", "ontime",
    "customer_name", "supplier_name", "material_shipped", 
    "origin_location", "origin_location_latitude", "origin_location_longitude"
]]

In [13]:
#trip_details 6 unique cols
trip_details_df = df[["booking_id", "transportation_distance_(km)", "trip_start_date", "trip_end_date",
    "destination_location", 'destination_location_latitude', 'destination_location_longitude'
]]

In [14]:
# Vehicles Tables 10 unique cols
vehicles_df = df[[
    "booking_id", "vehicle_registration", "vehicle_type", "minimum_kms_to_be_covered_in_a_day", 
    "driver_name", "driver_mobile_no", "gps_provider",
    "current_location", "current_location_latitude", "current_location_longitude", "data_ping_time"
]]

# Data Cleaning

In [15]:
booking_df

,booking_id,shipment_type,booking_date,planned_eta,actual_eta,ontime,customer_name,supplier_name,material_shipped,origin_location,origin_location_latitude,origin_location_longitude
0,AEIBK2027469,Regular,2020-08-26,2020-08-30,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Oms Logistics Pvt Ltd,Regulator - 12v,"Shive, Pune, Maharashtra",18.750621,73.877190
1,VCV00014153/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
2,VCV00014063/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
3,VCV00014741/082021,Regular,2020-08-28,2020-09-01,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Namakkal Sri Anjinaya Transport,Lu Hood Lock / Rh,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
4,AEIBK2027446,Regular,2020-08-26,2020-08-30,2020-08-29,Yes,Ford India Private Limited,Sterling Translogistics Private Limited,Lu Latch / Pin,"Khorajnanoda, Ahmedabad, Gujarat",22.961777,72.094219
...,...,...,...,...,...,...,...,...,...,...,...,...
3580,AEIBK1909187,Regular,2019-12-20,2019-12-20,2019-12-22,No,Larsen & Toubro Limited,S.B.Transport Company,Yoke & Pole Assembly,"Okhla industrial estate, south delhi, delhi",28.542809,77.280521
3581,AEIBK1907026,Regular,2019-11-29,2019-11-30,2019-12-18,No,Larsen & Toubro Limited,Sunita Carriers Private Limited,Rectifier Assembly,"Kuldanga, howrah, west bengal",16.560192,80.792293
3582,AEIBK1906955,Regular,2019-11-30,2019-11-30,2019-12-11,No,Larsen & Toubro Limited,Balaji Freight Carrier,Grs Starter,"Changodar, ahmedabad, gujarat",16.560192,80.792293
3583,WDSBKTP49392,Regular,2019-06-10,1899-12-30,1899-12-30,Yes,Larsen & Toubro Limited,Sunita Carriers Private Limited,Fxuwb-Lid Cover,"Sonai, kolkata, west bengal",23.525268,87.264424


In [16]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   booking_id                 3585 non-null   object 
 1   shipment_type              3585 non-null   object 
 2   booking_date               3585 non-null   object 
 3   planned_eta                3585 non-null   object 
 4   actual_eta                 3560 non-null   object 
 5   ontime                     3585 non-null   object 
 6   customer_name              3585 non-null   object 
 7   supplier_name              3585 non-null   object 
 8   material_shipped           3585 non-null   object 
 9   origin_location            3585 non-null   object 
 10  origin_location_latitude   3585 non-null   float64
 11  origin_location_longitude  3585 non-null   float64
dtypes: float64(2), object(10)
memory usage: 336.2+ KB


In [17]:
booking_df['ontime'].unique()

array(['Yes', 'No'], dtype=object)

In [18]:
booking_df['shipment_type'].unique()

array(['Regular', 'Market'], dtype=object)

In [19]:
booking_df['supplier_name'].unique()

array(['Oms Logistics Pvt Ltd', 'Vj Logistics',
       'Namakkal Sri Anjinaya Transport',
       'Sterling Translogistics Private Limited', 'Gayathri Transport',
       'Virs Tempo Service', 'Arvinth Transport', 'Eswar Transport',
       'Pratiksha Freight Carriers', 'Annai Velankanni Transport',
       'Vinayaka Roadlines', 'Susee Transporter', 'Archana Transport',
       'Amal Transport & Travels', 'Efc Infrastructures Private Limited',
       'Krc Logistics', 'Sr Transports', 'Ots Limited',
       'Tanaji Raosaheb Kolekar', 'A.V.Transports', 'Aakash Logistics',
       'Shree Om Transport Company', 'Unknown', 'Sundar',
       'Pawan R Logistics', 'Sri Pachiamman Transport',
       'Ekta Transport Company', 'Sam Express', 'Pachaiyammal Roadways',
       'Ekta Travels', 'Kasam Transport Service', 'Samandar Cargo',
       'S R Logistics', 'Vijay Transport', 'New Chennai Royal Transport',
       'Ctc Translogistic Pvt Ltd', 'Patanjali Parivahan Private Limited',
       'Sree Deepam Trans

In [20]:
booking_df['material_shipped'].unique()

array(['Regulator - 12v', 'Valve Spring', 'Lu Hood Lock / Rh',
       'Lu Latch / Pin', 'Planetary Axle', 'Spring Shackle Bolt',
       'Spring Shackle / Pin', 'Connecting Pin / Swivelling Lever',
       'Mounting Bracket / Round Bar', 'Centering Bolt',
       'Steuergeraet Motormanagement / Mdt Bs6',
       'Lu High-Pressure Transfer Pump For Commo', 'Rectangular Ring',
       'Lu End-Outline Marker Lamp / Led', '7mm Hole-Plate',
       'Spl Washer ( C1110 )', 'Rubber Buffer', 'Washer-3mm-Cab Mtg-2214',
       'Set Screw M8x1.25x35', 'Set Screw M12x1.75x35',
       'Plain Nut,M8x1.25', 'M16 Nyloc Nut', 'Spring Washer,M6', 'Washer',
       'Auto Parts', 'Empty Trays', 'Solenoid Fixing Screw',
       'Rectifier Assembly(12v-45a)', 'Motor Assembly - 26071474',
       'Switch Assy 7m14', 'Solenoid Assy-Sm76 Bare', 'F4d01700',
       'X0483110', 'F2p00119', 'Cl Optg Lever', 'Moulder Hose Kit',
       'Bracket / Lh Rr Suspension', 'Synchronizer Body',
       'Inner Synchronizer Ring / 2. Ga

In [21]:
booking_df['customer_name'].unique()

array(['Daimler India Commercial Vehicles Pvt Lt',
       'Ford India Private Limited', 'Ashok Leyland Limited',
       'Hero Motocorp Ltd', 'Lucas Tvs Ltd',
       'Tractors & Farm Equipment Limited', 'Brakes India Private Ltd',
       'Tvs Motor Company Limited',
       'Praveen Engineering Products India Pvt L',
       'Ericsson India Private Limited',
       'Otis Elevator Company (India) Ltd', 'Motherson Sumi System Ltd',
       'Jay Bharat Maruti Limited', 'Tvs Srichakra Limited',
       'Wabco India Ltd', 'Spicer India Private Ltd',
       'Valeo India Private Ltd', 'Larsen & Toubro Limited',
       'Getrag Transmissions India Pvt Ltd', 'Flyjac Logistics Pvt Ltd',
       'National Engg.Ind.Ltd.', 'Turbo Energy Pvt Ltd',
       'Cummins Technologies India Pvt Ltd', 'G-Tekt India Pvt Ltd.',
       'Hi-Tech Gears Ltd', 'Rico Auto Industries Ltd',
       'Talbros Automotive Components Ltd', 'Wipro Enterprises Pvt Ltd',
       'Comstar Automotive Technologies Pvt Ltd',
       'Kone E

In [22]:
booking_df

,booking_id,shipment_type,booking_date,planned_eta,actual_eta,ontime,customer_name,supplier_name,material_shipped,origin_location,origin_location_latitude,origin_location_longitude
0,AEIBK2027469,Regular,2020-08-26,2020-08-30,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Oms Logistics Pvt Ltd,Regulator - 12v,"Shive, Pune, Maharashtra",18.750621,73.877190
1,VCV00014153/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
2,VCV00014063/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
3,VCV00014741/082021,Regular,2020-08-28,2020-09-01,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Namakkal Sri Anjinaya Transport,Lu Hood Lock / Rh,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
4,AEIBK2027446,Regular,2020-08-26,2020-08-30,2020-08-29,Yes,Ford India Private Limited,Sterling Translogistics Private Limited,Lu Latch / Pin,"Khorajnanoda, Ahmedabad, Gujarat",22.961777,72.094219
...,...,...,...,...,...,...,...,...,...,...,...,...
3580,AEIBK1909187,Regular,2019-12-20,2019-12-20,2019-12-22,No,Larsen & Toubro Limited,S.B.Transport Company,Yoke & Pole Assembly,"Okhla industrial estate, south delhi, delhi",28.542809,77.280521
3581,AEIBK1907026,Regular,2019-11-29,2019-11-30,2019-12-18,No,Larsen & Toubro Limited,Sunita Carriers Private Limited,Rectifier Assembly,"Kuldanga, howrah, west bengal",16.560192,80.792293
3582,AEIBK1906955,Regular,2019-11-30,2019-11-30,2019-12-11,No,Larsen & Toubro Limited,Balaji Freight Carrier,Grs Starter,"Changodar, ahmedabad, gujarat",16.560192,80.792293
3583,WDSBKTP49392,Regular,2019-06-10,1899-12-30,1899-12-30,Yes,Larsen & Toubro Limited,Sunita Carriers Private Limited,Fxuwb-Lid Cover,"Sonai, kolkata, west bengal",23.525268,87.264424


## Delete outliers from all three tables

In [23]:
booking_df['planned_eta'].sort_values().tail(5)

1850    2020-10-07
2393    2020-10-07
2394    2020-10-08
2395    2020-11-07
1666    2020-12-05
Name: planned_eta, dtype: object

In [24]:
booking_df[booking_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502'])]


,booking_id,shipment_type,booking_date,planned_eta,actual_eta,ontime,customer_name,supplier_name,material_shipped,origin_location,origin_location_latitude,origin_location_longitude
3583,WDSBKTP49392,Regular,2019-06-10,1899-12-30,1899-12-30,Yes,Larsen & Toubro Limited,Sunita Carriers Private Limited,Fxuwb-Lid Cover,"Sonai, kolkata, west bengal",23.525268,87.264424
3584,WDSBKTP44502,Regular,2019-04-15,1899-12-30,1899-12-30,No,Larsen & Toubro Limited,A S Transports,Insrt-Vlv St Intk,"Mugabala, bangalore rural, karnataka",16.560192,80.792293


In [25]:
rows_to_keep = ~booking_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502'])
booking_df = booking_df[rows_to_keep]

In [26]:
booking_df.shape[0]

3583

In [27]:
rows_to_keep = ~trip_details_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502'])
trip_details_df = booking_df[rows_to_keep]

In [28]:

trip_details_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502']).value_counts()

booking_id
False    3583
Name: count, dtype: int64

In [29]:
rows_to_keep = ~vehicles_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502'])
vehicles_df = vehicles_df[rows_to_keep]

In [30]:
vehicles_df['booking_id'].isin(['WDSBKTP49392', 'WDSBKTP44502']).value_counts()

booking_id
False    3583
Name: count, dtype: int64

## Create a State Columns

### booking table

In [31]:
booking_df.columns

Index(['booking_id', 'shipment_type', 'booking_date', 'planned_eta',
       'actual_eta', 'ontime', 'customer_name', 'supplier_name',
       'material_shipped', 'origin_location', 'origin_location_latitude',
       'origin_location_longitude'],
      dtype='object')

In [32]:
booking_df['origin_location'].unique()

array(['Shive, Pune, Maharashtra',
       'Daimler India Commercial Vehicles,Kanchipuram,Tamil Nadu',
       'Khorajnanoda, Ahmedabad, Gujarat',
       'Naickenpalayam, Coimbatore, Tamil Nadu',
       'Heromotocorp Ltd,Alwar,Rajasthan',
       'Jagadambigainagar, Tiruvallur, Tamil Nadu',
       'Ashok Leyland Ennore,Chennai,Tamil Nadu',
       'Tractors & Farm Equipment Limited,Kanchipuram,Tamil Nadu',
       'Kalri, Mahesana, Gujarat',
       'Lucas Tvs Ltd-Pondy,Pondy,Pondicherry',
       'Lucas Tvs Ltd-Ambattur,Chennai,Tamil Nadu',
       'Tvslsl Hub (Chakan),Pune,Maharashtra',
       'Praveen Engineering Industries,Hosur,Tamil Nadu',
       'Jamalpur, Gurgaon, Haryana', 'Anekal, Bangalore, Karnataka',
       'Ashok Leyland Plant 2-Hosur,Hosur,Karnataka',
       'Motherson Sumi System Ltd,Ahmedabad,Gujarat',
       'Ashok Leyland Plant 1- Hosur,Hosur,Karnataka',
       'Jay Bharat Maruti Ltd,Ahmedabad,Gujarat',
       'Pozhal, Tiruvallur, Tamil Nadu',
       'Gopanapalli, Krishnagir

In [33]:
booking_df['origin_states'] = booking_df['origin_location'].str.split(',').str[-1].str.strip()

In [34]:
booking_df['origin_states'].unique()

array(['Maharashtra', 'Tamil Nadu', 'Gujarat', 'Rajasthan', 'Pondicherry',
       'Haryana', 'Karnataka', 'Uttar Pradesh', 'West Bengal', 'Odisha',
       'Jharkhand', 'Bihar', 'maharashtra', 'tamil nadu', 'haryana',
       'karnataka', 'pondicherry', 'assam', 'gujarat', 'bihar',
       'andhra pradesh', 'odisha', 'west bengal', 'uttar pradesh',
       'telangana', 'rajasthan', 'jharkhand', 'chattisgarh', 'delhi',
       'kerala'], dtype=object)

In [35]:
state_mapping = {
    'maharashtra': 'Maharashtra',
    'gujarat': 'Gujarat',
    'goa': 'Goa',
    'andhra pradesh': 'Andhra Pradesh',
    'tamil nadu': 'Tamil Nadu',
    'rajasthan': 'Rajasthan',
    'pondicherry': 'Pondicherry',
    'haryana': 'Haryana',
    'karnataka': 'Karnataka',
    'uttar radesh': 'Uttar Pradesh',  
    'west bengal': 'West Bengal',
    'odisha': 'Odisha',
    'jharkhand': 'Jharkhand',
    'bihar': 'Bihar',
    'telangana': 'Telangana', 
    'kerala': 'Kerala',
    'punjab': 'Punjab',
    'madhya pradesh': 'Madhya Pradesh',
    'assam': 'Assam',
    'delhi': 'Delhi',
    
}

In [36]:
booking_df['origin_states'] = booking_df['origin_states'].replace(state_mapping)

In [36]:
booking_df['origin_states'].value_counts()

origin_states
Tamil Nadu        1275
Maharashtra        492
Gujarat            339
Karnataka          331
Haryana            267
West Bengal        160
Pondicherry        121
Odisha             106
Assam               99
Telangana           94
uttar pradesh       76
Jharkhand           63
Delhi               39
Rajasthan           36
chattisgarh         28
Andhra Pradesh      27
Bihar               17
Uttar Pradesh       11
Kerala               2
Name: count, dtype: int64

### trip table

In [40]:
trip_details_df.columns

Index(['booking_id', 'transportation_distance_(km)', 'trip_start_date',
       'trip_end_date', 'destination_location',
       'destination_location_latitude', 'destination_location_longitude'],
      dtype='object')

In [39]:
#trip_details 6 unique cols
trip_details_df = df[["booking_id", "transportation_distance_(km)", "trip_start_date", "trip_end_date",
    "destination_location", 'destination_location_latitude', 'destination_location_longitude'
]]

In [41]:
trip_details_df['destination_states'] = trip_details_df['destination_location'].str.split(',').str[-1].str.strip()

In [42]:
trip_details_df['destination_states'].value_counts()

destination_states
Tamil Nadu                    1241
Haryana                        372
Gujarat                        329
Maharashtra                    279
Karnataka                      257
West Bengal                    175
Rajasthan                      126
Odisha                         123
Telangana                      113
Uttar Pradesh                   99
Assam                           97
Pondicherry                     82
Jharkhand                       82
Andhra Pradesh                  62
Delhi                           53
Himachal Pradesh                44
Bihar                           15
Meghalaya                        8
Punjab                           8
Jammu & Kashmir                  6
Chattisgarh                      4
Kerala                           3
Madhya Pradesh                   2
Uttarakhand                      2
Central Development Region       1
Chandigarh                       1
Dadra & Nagar Haveli             1
Name: count, dtype: int64

### vehicles

In [43]:
vehicles_df['current_location'].value_counts()

current_location
Perumalpattu - Kottamedu Rd, Oragadam Industrial Corridor, Vattambakkam R.F., Tamil Nadu 631605, India                             198
Unnamed Road, Chharodi, Gujarat 382170, India                                                                                      101
Unnamed Road, Maraimalai Nagar, Tamil Nadu 603209, India                                                                            96
Singaperumal Koil - Sriperumbudur Rd, Oragadam Industrial Corridor, Vattambakkam R.F., Tamil Nadu 631605, India                     79
Block No. 1 Survey No. 466,467,468,469, Plot No. 474, 485, Hosur - Thally Rd, Near By Karnoor Village, Tamil Nadu 635110, India     52
                                                                                                                                  ... 
Gj Sh 67, Kumbhale, Maharashtra 422208, India                                                                                        1
Unnamed Road, Gujarat 391243, India   

In [44]:
vehicles_df['current_location'].str.split(',').str[-2].str.strip().str.split().str[0] 

0             Tamil
1             Tamil
2             Tamil
3             Tamil
4       Maharashtra
           ...     
3578          Delhi
3579          Delhi
3580          Delhi
3581           West
3582        Gujarat
Name: current_location, Length: 3583, dtype: object

In [45]:
def extract_state(location):
    match = re.match(r"([a-zA-Z ]+)", location) # Matches one or more words(a-z, A-Z, and spaces) from the beginning of the string.
    if match:
        return match.group(1).strip()  # Return the matched state name (group 1), removing extra spaces
    return None  # Or handle cases where no state is found differently (e.g., return "Unknown")

In [46]:
vehicles_df['current_states'] =  vehicles_df['current_location'].str.split(',').str[-2].str.strip()

In [47]:
vehicles_df['current_states']  = vehicles_df['current_states'].str.strip().str.extract(r'([A-Za-z\s]+)')

In [48]:
vehicles_df['current_states'].unique()

array(['Tamil Nadu ', 'Maharashtra ', 'Haryana ', 'Puducherry ',
       'Uttar Pradesh ', 'Karnataka ', 'Gujarat ', 'Unnamed Road',
       'Rajasthan ', 'Himachal Pradesh ', 'Madhya Pradesh ',
       'Andhra Pradesh ', 'Assam ', 'Tamil Nadu', 'West Bengal ',
       'Bihar ', 'Odisha ', 'Gujarat', 'Karnataka', 'Jharkhand ',
       'Telangana ', 'Punjab ', nan, 'Telangana', 'Meghalaya ',
       'Chhattisgarh ', 'Uttarakhand ', 'Rajasthan', ' ', 'Maharashtra',
       'Delhi ', 'Kerala ', 'Sedarapet'], dtype=object)

# trip tables cleaning 


In [ ]:
trip_details_df

,booking_id,transportation_distance_(km),trip_start_date,trip_end_date,destination_location,destination_location_latitude,destination_location_longitude,destination_states
0,AEIBK2027469,1290.0,2020-08-26,2020-08-28,"Pondur, Kanchipuram, Tamil Nadu",12.930429,79.931163,Tamil Nadu
1,VCV00014153/082021,29.0,2020-08-27,2020-08-27,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
2,VCV00014063/082021,21.0,2020-08-27,2020-08-27,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
3,VCV00014741/082021,20.0,2020-08-28,2020-08-28,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
4,AEIBK2027446,1900.0,2020-08-26,2020-08-28,"Singaperumalkoil, Kanchipuram, Tamil Nadu",12.786517,79.975221,Tamil Nadu
...,...,...,...,...,...,...,...,...
3580,AEIBK1909187,25.0,2019-12-20,2020-01-16,"Palam Airport, South West Delhi, Delhi",28.553908,77.090981,Delhi
3581,AEIBK1907026,30.0,2019-11-29,2020-02-05,"Kuldanga, Howrah, West Bengal",22.593221,88.472233,West Bengal
3582,AEIBK1906955,25.0,2019-11-30,2020-01-02,"Zanu, Ahmedabad, Gujarat",23.042185,72.697893,Gujarat
3583,WDSBKTP49392,51.0,1899-12-30,1899-12-30,"Kalyani, Nadia, West Bengal",22.952176,88.457141,West Bengal


In [ ]:
booking_df[['origin_location', 'origin_location_latitude',
       'origin_location_longitude', 'origin_states']]

,origin_location,origin_location_latitude,origin_location_longitude,origin_states
0,"Shive, Pune, Maharashtra",18.750621,73.877190,Maharashtra
1,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
2,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
3,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000,Tamil Nadu
4,"Khorajnanoda, Ahmedabad, Gujarat",22.961777,72.094219,Gujarat
...,...,...,...,...
3580,"Okhla industrial estate, south delhi, delhi",28.542809,77.280521,Delhi
3581,"Kuldanga, howrah, west bengal",16.560192,80.792293,West Bengal
3582,"Changodar, ahmedabad, gujarat",16.560192,80.792293,Gujarat
3583,"Sonai, kolkata, west bengal",23.525268,87.264424,West Bengal


In [ ]:
trip_details_df[trip_details_df['transportation_distance_(km)'].isnull()]

,booking_id,transportation_distance_(km),trip_start_date,trip_end_date,destination_location,destination_location_latitude,destination_location_longitude,destination_states
2143,AEIBK2023118,NaN,2020-07-18,2020-07-21,"Sanaswadi, Pune, Maharashtra",18.674382,74.194453,Maharashtra
2148,AEIBK2022783,NaN,2020-07-14,2020-07-21,"Dundahera, Gurgaon, Haryana",28.512760,77.078050,Haryana
2160,AEIBK2023123,NaN,2020-07-18,2020-07-21,"Sanaswadi, Pune, Maharashtra",18.674382,74.194453,Maharashtra
2166,AEIBK2022478,NaN,2020-07-11,2020-08-11,"Gurgaon, Gurgaon, Haryana",28.430086,77.017841,Haryana
2167,AEIBK2022465,NaN,2020-07-10,2020-07-22,"Raipur Kutchery, Raipur, Chattisgarh",21.237547,81.630908,Chattisgarh
...,...,...,...,...,...,...,...,...
2893,AEIBK2019969,NaN,2020-06-07,2020-06-12,"Mathura Road Faridabad, Faridabad, Haryana",28.463698,77.333878,Haryana
2897,AEIBK2020007,NaN,2020-06-08,2020-06-10,"Onnalvadi, Krishnagiri, Tamil Nadu",12.683589,77.859239,Tamil Nadu
2898,AEIBK2020021,NaN,2020-06-01,2020-06-12,"Jamalpur, Gurgaon, Haryana",28.373519,76.835337,Haryana
2900,AEIBK2019796,NaN,2020-06-01,2020-06-12,"Jamalpur, Gurgaon, Haryana",28.373519,76.835337,Haryana


In [ ]:
vehicles_df

,booking_id,vehicle_registration,vehicle_type,minimum_kms_to_be_covered_in_a_day,driver_name,driver_mobile_no,gps_provider,current_location,current_location_latitude,current_location_longitude,data_ping_time,current_states
0,AEIBK2027469,MH14GD9464,32 FT Multi-Axle 14MT - HCV,NaN,VIRAT NILAPALLE,9.960008e+09,Consent Track,"Singaperumal Koil - Sriperumbudur Rd, Oragadam...",12.837127,79.951866,2020-08-28,Tamil Nadu
1,VCV00014153/082021,TN30BC9320,NaN,NaN,SENTHIL KUMAR,NaN,Vamosys,"Gokulapuram Main Rd, Maraimalai Nagar, Tamil N...",12.768090,80.026660,2020-08-28,Tamil Nadu
2,VCV00014063/082021,TN30BB1036,NaN,NaN,ANBU,NaN,Vamosys,"Singaperumal Koil - Sriperumbudur Rd, Oragadam...",12.835262,79.952533,2020-08-28,Tamil Nadu
3,VCV00014741/082021,TN88D4134,NaN,NaN,SUDHAKAR,NaN,Vamosys,"Unnamed Road, Oragadam Industrial Corridor, Or...",12.839172,79.951355,2020-08-28,Tamil Nadu
4,AEIBK2027446,GJ01DZ8943,32 FT Single-Axle 7MT - HCV,NaN,MAN SINGH,6.396821e+09,Consent Track,"Rajmata Jijau Marg, Suhash Colony, Beed, Mahar...",18.996400,75.740200,2020-08-28,Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...,...
3580,AEIBK1909187,HR38S1605,40 FT 3XL Trailer 35MT,250.0,NaN,NaN,Consent Track,"Unnamed Road, Tekhand, Okhla Phase I, Okhla In...",28.523281,77.287640,2019-12-16,Delhi
3581,AEIBK1907026,NL01G0589,40 FT Flat Bed Double-Axle 21MT - Trailer,250.0,NaN,NaN,Consent Track,"45, Grand Trunk Rd, Naora, Shibpur, Howrah, We...",25.136247,82.566526,2019-12-12,West Bengal
3582,AEIBK1906955,GJ15UU7437,40 FT 3XL Trailer 35MT,250.0,MAHENDRA SINGH,8.949298e+09,Consent Track,"Unnamed Road, Gujarat 391243, India",22.190830,73.078550,2019-12-05,Gujarat
3583,WDSBKTP49392,WB59B9152,40 FT Flat Bed Multi-Axle 27MT - Trailer,NaN,NaN,NaN,Null,Null,NaN,NaN,NaN,NaN


In [49]:
booking_df.to_excel("Bookings.xlsx", index=False, sheet_name='Bookings')
print("Excel 'booking' files created successfully!")

Excel 'booking' files created successfully!


In [50]:
trip_details_df.to_excel("Trip_Details.xlsx", index=False, sheet_name='trip_details')
print("Excel 'trip_details' files created successfully!")

Excel 'trip_details' files created successfully!


In [51]:
vehicles_df.to_excel("Vehicles.xlsx", index=False, sheet_name='vehicles')
print("Excel 'vehicles' files created successfully!")

Excel 'vehicles' files created successfully!


# Create new small relevant Dataset

In [12]:
# booking Tables
booking_cols = [    
    "booking_id", "shipment_type", "booking_date", 
    "planned_eta", "actual_eta", "ontime",
    "customer_name", "supplier_name", "material_shipped", 
    "origin_location", "origin_location_latitude", "origin_location_longitude"
]

df[booking_cols].to_excel("Bookings.xlsx", index=False, sheet_name='Bookings')
print("Excel 'booking' files created successfully!")



Excel 'booking' files created successfully!


In [13]:
# Trip Details Tables
trip_details_cols = [
    "booking_id", 
    "transportation_distance_(km)",
    "trip_start_date", "trip_end_date",
    "destination_location", 'destination_location_latitude', 'destination_location_longitude',
]

df[trip_details_cols].to_excel("Trip_Details.xlsx", index=False, sheet_name='trip_details')
print("Excel 'trip_details' files created successfully!")


Excel 'trip_details' files created successfully!


In [14]:
# Vehicles Tables
vehicles_cols = [
    "booking_id", "vehicle_registration", "vehicle_type", "minimum_kms_to_be_covered_in_a_day", 
    "driver_name", "driver_mobile_no", "gps_provider",
    "current_location", "current_location_latitude", "current_location_longitude", "data_ping_time"
]

df[vehicles_cols].to_excel("Vehicles.xlsx", index=False, sheet_name='vehicles')
print("Excel 'vehicles' files created successfully!")

Excel 'vehicles' files created successfully!


In [54]:
df[booking_cols]

,booking_id,shipment_type,booking_date,planned_eta,actual_eta,ontime,customer_name,supplier_name,material_shipped,origin_location,origin_location_latitude,origin_location_longitude
0,AEIBK2027469,Regular,2020-08-26,2020-08-30,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Oms Logistics Pvt Ltd,Regulator - 12v,"Shive, Pune, Maharashtra",18.750621,73.877190
1,VCV00014153/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
2,VCV00014063/082021,Regular,2020-08-27,2020-08-31,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Vj Logistics,Valve Spring,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
3,VCV00014741/082021,Regular,2020-08-28,2020-09-01,2020-08-28,Yes,Daimler India Commercial Vehicles Pvt Lt,Namakkal Sri Anjinaya Transport,Lu Hood Lock / Rh,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
4,AEIBK2027446,Regular,2020-08-26,2020-08-30,2020-08-29,Yes,Ford India Private Limited,Sterling Translogistics Private Limited,Lu Latch / Pin,"Khorajnanoda, Ahmedabad, Gujarat",22.961777,72.094219
...,...,...,...,...,...,...,...,...,...,...,...,...
3580,AEIBK1909187,Regular,2019-12-20,2019-12-20,2019-12-22,No,Larsen & Toubro Limited,S.B.Transport Company,Yoke & Pole Assembly,"Okhla industrial estate, south delhi, delhi",28.542809,77.280521
3581,AEIBK1907026,Regular,2019-11-29,2019-11-30,2019-12-18,No,Larsen & Toubro Limited,Sunita Carriers Private Limited,Rectifier Assembly,"Kuldanga, howrah, west bengal",16.560192,80.792293
3582,AEIBK1906955,Regular,2019-11-30,2019-11-30,2019-12-11,No,Larsen & Toubro Limited,Balaji Freight Carrier,Grs Starter,"Changodar, ahmedabad, gujarat",16.560192,80.792293
3583,WDSBKTP49392,Regular,2019-06-10,1899-12-30,1899-12-30,Yes,Larsen & Toubro Limited,Sunita Carriers Private Limited,Fxuwb-Lid Cover,"Sonai, kolkata, west bengal",23.525268,87.264424


In [55]:
df[trip_details_cols]

,booking_id,transportation_distance_(km),trip_start_date,trip_end_date,destination_location,destination_location_latitude,destination_location_longitude
0,AEIBK2027469,1290.0,2020-08-26,2020-08-28,"Pondur, Kanchipuram, Tamil Nadu",12.930429,79.931163
1,VCV00014153/082021,29.0,2020-08-27,2020-08-27,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
2,VCV00014063/082021,21.0,2020-08-27,2020-08-27,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
3,VCV00014741/082021,20.0,2020-08-28,2020-08-28,"Daimler India Commercial Vehicles,Kanchipuram,...",12.839000,79.954000
4,AEIBK2027446,1900.0,2020-08-26,2020-08-28,"Singaperumalkoil, Kanchipuram, Tamil Nadu",12.786517,79.975221
...,...,...,...,...,...,...,...
3580,AEIBK1909187,25.0,2019-12-20,2020-01-16,"Palam Airport, South West Delhi, Delhi",28.553908,77.090981
3581,AEIBK1907026,30.0,2019-11-29,2020-02-05,"Kuldanga, Howrah, West Bengal",22.593221,88.472233
3582,AEIBK1906955,25.0,2019-11-30,2020-01-02,"Zanu, Ahmedabad, Gujarat",23.042185,72.697893
3583,WDSBKTP49392,51.0,1899-12-30,1899-12-30,"Kalyani, Nadia, West Bengal",22.952176,88.457141
